## 1. 검색 성능 평가
- 구축한 검색 시스템(Retriever)이 얼마나 잘 동작하는지 정량적으로 평가하는 것은 매우 중요함. 
- 이를 위해 테스트 데이터셋을 만들고, 정보 검색(IR) 분야의 평가지표를 사용함.

### 1.1 테스트 데이터 준비

- 좋은 평가를 위해서는 양질의 테스트 데이터셋이 필요함. 
- 여기서는 기존 문서들을 기반으로 질문-답변(QA) 쌍을 합성하고, 이를 검토/수정하여 사용함.

`(1) 평가용 문서 데이터 정제`

- 이전에 사용한 `korean_docs`를 평가용으로 좀 더 가공함.
  - 각 문서에 고유 `doc_id` 메타데이터 추가.
  - 문장 구분 기호를 줄바꿈 문자로 변경 (가독성 및 LLM 처리 용이성).
  - 문서 내용 끝에 해당 문서가 어떤 기업에 대한 정보인지 명시 (LLM이 QA 생성 시 참고하도록).
- JSONL (JSON Lines) 형식으로 저장하여 재사용 용이하게 함.
- **장점:** 평가 데이터의 일관성 및 재현성 확보, LLM이 QA 생성 시 컨텍스트 명확화.
- **단점:** 수동 전처리 과정 필요, 데이터 형식 변환에 따른 약간의 오버헤드.

In [1]:
from dotenv import load_dotenv
load_dotenv() 
from glob import glob
import os
import json

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import CharacterTextSplitter
from transformers import AutoTokenizer

c:\Users\ryusg\anaconda3\envs\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 데이터 로드 함수
def load_text_files(txt_files):
    data = []
    for text_file in txt_files:
        print(f"로딩 중: {text_file}")
        # encoding='utf-8' 명시하여 한글 파일 로드 문제 방지
        loader = TextLoader(text_file, encoding='utf-8') 
        data.extend(loader.load())
    return data

In [4]:
korean_txt_files = glob(os.path.join('data', '*_KR.txt'))
if not korean_txt_files:
    print("'data' 폴더에 '*_KR.txt' 파일이 없습니다. 예시 데이터를 생성하거나 경로를 확인하세요.")
    korean_data = [] # 빈 리스트로 초기화
else:
    korean_data = load_text_files(korean_txt_files)

print(f"\n로드된 전체 문서 수: {len(korean_data)}")
if korean_data:
    print(f"첫 번째 문서 내용 일부: {korean_data[0].page_content[:100]}...")


로딩 중: data\Rivian_KR.txt
로딩 중: data\Tesla_KR.txt

로드된 전체 문서 수: 2
첫 번째 문서 내용 일부: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당...


In [5]:
# Hugging Face 임베딩 모델(BAAI/bge-m3)이 사용하는 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained("BAAI/bge-m3")

# 문장 구분자(정규식)를 사용하여 텍스트 분할기 생성
text_splitter = CharacterTextSplitter.from_huggingface_tokenizer(
    tokenizer=tokenizer,          # 토큰 수 계산 기준
    separator=r"[.!?]\s+",     # 문장 구분자: 마침표, 느낌표, 물음표 뒤 공백
    chunk_size=100,             # 청크 최대 토큰 수 (토크나이저 기준)
    chunk_overlap=20,            # 청크 간 중복 토큰 수
    is_separator_regex=True,    # separator가 정규식임을 명시
    keep_separator=True,        # 구분자 유지 여부
)

korean_docs = []
if korean_data: 
    korean_docs = text_splitter.split_documents(korean_data)

print(f"\n분할된 한국어 문서 수: {len(korean_docs)}")
if korean_docs:
    print(f"첫 번째 분할 문서 내용: {korean_docs[0].page_content}")
    print(f"첫 번째 분할 문서 메타데이터: {korean_docs[0].metadata}")


분할된 한국어 문서 수: 8
첫 번째 분할 문서 내용: 2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다
첫 번째 분할 문서 메타데이터: {'source': 'data\\Rivian_KR.txt'}


In [6]:
final_docs_for_eval = []
if korean_docs:
    for i, doc in enumerate(korean_docs):
        new_doc = doc.copy()
        # metadata에 doc_id 추가
        new_doc.metadata['doc_id'] = f'eval_doc_{i}'
        
        # metadata에서 기업 이름 정보 추출하여 page_content에 추가 (LLM QA 생성 시 참고용)
        # 예시: 'data/리비안_KR.txt' -> '리비안'
        source_filename = os.path.basename(new_doc.metadata.get('source', ''))
        corp_name = source_filename.split('_')[0] if '_' in source_filename else source_filename.replace('.txt', '')
        new_doc.page_content = f"{new_doc.page_content}\n\n(참고: 이 문서는 '{corp_name}'에 대한 정보를 담고 있습니다.)"
        final_docs_for_eval.append(new_doc)

    print(f"평가용으로 정제된 문서 수: {len(final_docs_for_eval)}")
    if final_docs_for_eval:
        print("\n첫 번째 정제 문서 예시:")
        print(final_docs_for_eval[0].page_content)
        print("-" * 30)
        print(final_docs_for_eval[0].metadata)
        print("=" * 30)

    with open('./data/final_docs_for_eval.jsonl', 'w', encoding='utf-8') as f:
        for doc_item in final_docs_for_eval:
            # Document 객체를 dict로 변환 후 JSON 문자열로 저장
            # metadata의 source 경로가 \로 되어있으면 json.dumps 시 \\로 이스케이프됨. 이는 정상.
            f.write(json.dumps({'page_content': doc_item.page_content, 'metadata': doc_item.metadata}) + '\n')
    print("\n평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.")

else:
    print("korean_docs가 없어 평가용 문서를 정제할 수 없습니다.")

평가용으로 정제된 문서 수: 8

첫 번째 정제 문서 예시:
2009년 MIT 박사 과정생 RJ 스캐린지가 설립한 리비안(Rivian)은 혁신적인 미국 전기차 제조업체입니다. 2011년부터 자율주행 전기차에 집중했던 리비안은 2015년 상당한 투자를 통해 비약적인 성장을 거듭하며 미시간과 베이 지역에 연구 시설을 설립했습니다. 주요 공급업체와의 거리를 좁히기 위해 본사를 미시간주 리보니아로 이전했습니다

(참고: 이 문서는 'Rivian'에 대한 정보를 담고 있습니다.)
------------------------------
{'source': 'data\\Rivian_KR.txt', 'doc_id': 'eval_doc_0'}

평가용 문서가 './data/final_docs_for_eval.jsonl' 파일로 저장되었습니다.


C:\Users\ryusg\AppData\Local\Temp\ipykernel_9764\2009137773.py:4: PydanticDeprecatedSince20: The `copy` method is deprecated; use `model_copy` instead. See the docstring of `BaseModel.copy` for details about how to handle `include` and `exclude`. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  new_doc = doc.copy()


`(2) Question-Answer (QA) 합성`

- 정제된 문서(`final_docs_for_eval`)의 각 청크(문서)를 컨텍스트로 하여, LLM(여기서는 OpenAI GPT 모델)을 사용해 질문과 답변 쌍을 생성함.
- `PydanticOutputParser`를 사용하여 LLM의 출력을 구조화된 객체(QAPair, QASet)로 파싱함.
- 프롬프트 엔지니어링을 통해 사실 기반의, 검색 엔진 사용자 스타일 질문을 생성하도록 유도함.
- **장점:** 대량의 QA 평가 데이터셋을 자동으로 생성 가능.
- **단점:**
  - LLM API 사용 비용 발생 (OpenAI API 키 필요).
  - 생성된 QA의 품질이 완벽하지 않을 수 있어 검토 및 수정 과정이 필요함.
  - 프롬프트에 민감하게 반응하므로, 원하는 품질의 QA를 얻기 위해 프롬프트 튜닝이 필요할 수 있음.

In [7]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from langchain_core.pydantic_v1 import BaseModel, Field # Pydantic V1 사용
from typing import List
import pandas as pd

# QA 쌍을 위한 Pydantic 모델 정의
class QAPair(BaseModel):
    question: str = Field(description="생성된 질문 (한국어로 작성)")
    answer: str = Field(description="질문에 대한 답변 (한국어로 작성, 질문의 핵심 내용을 반영)")

class QASet(BaseModel):
    qa_pairs: List[QAPair] = Field(description="질문-답변 쌍(QAPair)의 리스트")

# QA 생성 프롬프트 템플릿
QA_GENERATION_TEMPLATE_KOREAN = """
당신은 주어진 컨텍스트를 기반으로 사실에 입각한 질문-답변 쌍을 생성하는 AI입니다.
컨텍스트에서 특정하고 간결한 사실 정보로 답변할 수 있는 질문 {num_questions_per_chunk}개를 만드세요.
질문은 사용자가 검색 엔진에 질문할 법한 스타일로 작성해주세요.
질문에 "제시된 구절에 따르면" 또는 "컨텍스트에 따르면" 같은 문구는 포함하지 마세요.
답변에는 명확하고 완전한 정보를 제공하기 위해 질문의 핵심 내용이 포함되도록 하세요.

---------------------------------------------------------
출력은 다음 형식으로 제공해주세요:
{format_instructions}
---------------------------------------------------------
이제 컨텍스트를 제공합니다:
컨텍스트: {context}
"""

# ChatOpenAI 모델 초기화 (환경변수 OPENAI_API_KEY 설정 필요)
qa_llm = ChatOpenAI(
    model="gpt-4o-mini", # 또는 다른 gpt 모델 (예: gpt-3.5-turbo)
    max_tokens=1000, # 답변 최대 토큰 수 조절
    temperature=0.2,  # 낮은 온도로 사실 기반 생성 유도
)

pydantic_parser_qa = PydanticOutputParser(pydantic_object=QASet)

qa_generation_prompt = ChatPromptTemplate.from_template(
    template=QA_GENERATION_TEMPLATE_KOREAN,
    partial_variables={"format_instructions": pydantic_parser_qa.get_format_instructions()}
)


c:\Users\ryusg\anaconda3\envs\langchain_env\Lib\site-packages\IPython\core\interactiveshell.py:3549: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  exec(code_obj, self.user_global_ns, self.user_ns)


AttributeError: type object 'QASet' has no attribute 'model_json_schema'

In [ ]:
qa_generation_chain = qa_generation_prompt | qa_llm | pydantic_parser_qa

# QA 생성 테스트 (첫 번째 정제 문서 사용)
if final_docs_for_eval:
    test_context_for_qa = final_docs_for_eval[0].page_content
    print(f"QA 생성 테스트용 컨텍스트:\n{test_context_for_qa}\n")
    
    qa_set_example = qa_generation_chain.invoke({
        "context": test_context_for_qa,
        "num_questions_per_chunk": 1 # 테스트로 1개만 생성
    })
    print("생성된 QA 쌍 예시:")
    for qa_pair in qa_set_example.qa_pairs:
        print(f"  질문: {qa_pair.question}")
        print(f"  답변: {qa_pair.answer}")
    print("실제 QA 생성은 주석 처리됨 (API 비용 발생 방지). 필요시 주석 해제 후 실행.")
else:
    print("정제된 평가용 문서가 없어 QA 생성 테스트를 스킵합니다.")

In [ ]:
# 전체 평가용 문서에 대해 QA 생성 (실제 실행 시 시간 및 비용 소요)
NUM_QUESTIONS_PER_CHUNK = 2 # 각 문서 청크당 생성할 QA 개수
generated_qa_outputs = []

if final_docs_for_eval and False: # << 실제 실행 시 False를 True로 변경 >>
    print(f"{len(final_docs_for_eval)}개의 문서에 대해 QA 생성을 시작합니다...")
    for i, doc_item in enumerate(final_docs_for_eval):
        print(f"{i+1}/{len(final_docs_for_eval)}번째 문서 처리 중...", end=' ')
        try:
            qa_set_generated = qa_generation_chain.invoke({
                "context": doc_item.page_content,
                "num_questions_per_chunk": NUM_QUESTIONS_PER_CHUNK
            })
            for qa_pair in qa_set_generated.qa_pairs:
                generated_qa_outputs.append({
                    'context': [doc_item.page_content], # 정답 컨텍스트 (리스트 형태)
                    'source': [doc_item.metadata.get('source', '')], # 출처 (리스트 형태)
                    'doc_id': [doc_item.metadata.get('doc_id', '')], # 문서 ID (리스트 형태)
                    'question': qa_pair.question,
                    'answer': qa_pair.answer
                })
            print("성공")
        except Exception as e:
            print(f"실패: {e}")
            continue # 오류 발생 시 다음 문서로
    
    df_generated_qa_test = pd.DataFrame(generated_qa_outputs)
    print(f"\n총 {df_generated_qa_test.shape[0]}개의 QA 쌍이 생성되었습니다.")
    # df_generated_qa_test.to_excel("./data/generated_qa_test.xlsx", index=False)
    # print("생성된 QA 데이터가 './data/generated_qa_test.xlsx' 파일로 저장되었습니다.")
    # display(df_generated_qa_test.head())
else:
    print("QA 합성은 실제 실행되지 않았습니다. (final_docs_for_eval이 없거나, 실행 플래그가 False임)")
    # 예시로 빈 DataFrame 또는 미리 준비된 파일 로드
    # df_generated_qa_test = pd.DataFrame(columns=['context', 'source', 'doc_id', 'question', 'answer'])
    # 실습 편의를 위해, 원본 노트북의 qa_test_revised.xlsx 파일을 사용한다고 가정.
    # 해당 파일이 ./data/qa_test_revised.xlsx 에 있다고 가정.
    try:
        df_qa_test_for_eval = pd.read_excel("./data/qa_test_revised.xlsx")
        print("미리 준비된 './data/qa_test_revised.xlsx' 파일을 로드했습니다.")
        # display(df_qa_test_for_eval.head())
    except FileNotFoundError:
        print("'./data/qa_test_revised.xlsx' 파일을 찾을 수 없습니다. QA 생성 단계를 실행하거나 파일을 준비해주세요.")
        df_qa_test_for_eval = pd.DataFrame(columns=['context', 'source', 'doc_id', 'question', 'answer'])

`(3) 테스트 데이터 검토 및 수정`

- LLM이 생성한 QA 데이터는 완벽하지 않을 수 있으므로, 사람이 직접 검토하고 수정하는 과정이 중요함.
- (예시) `generated_qa_test.xlsx` 파일을 열어 질문의 적절성, 답변의 정확성, 컨텍스트와의 관련성 등을 확인하고 수정 후 `qa_test_revised.xlsx`로 저장하여 사용한다고 가정함.
- **장점:** 평가 데이터의 품질을 크게 향상시켜 평가 결과의 신뢰도를 높임.
- **단점:** 시간과 노력이 많이 소요되는 수동 작업임.

In [ ]:
# 검토 및 수정된 테스트 데이터셋 로드 (위 셀에서 이미 df_qa_test_for_eval로 로드 시도함)
if not df_qa_test_for_eval.empty:
    print(f"검토/수정된 QA 테스트 데이터셋 로드 완료. 총 {df_qa_test_for_eval.shape[0]}개 QA.")
    # display(df_qa_test_for_eval.head())
else:
    print("검토/수정된 QA 테스트 데이터셋이 로드되지 않았습니다.")

### 4.2 Information Retrieval (IR) 평가지표

- 검색 시스템이 얼마나 관련 있는 문서를 잘 찾아내는지, 그리고 그 순서는 적절한지를 평가하는 지표들임.
- `K-RAG` 패키지 (`!pip install krag`) 또는 직접 구현하여 사용 가능. 여기서는 `K-RAG` 패키지의 `OfflineRetrievalEvaluators` 사용 예시를 각색하여 설명함.
- 주요 지표: Hit Rate, MRR, mAP@k, NDCG@k 등.

#### 가상 평가 도구 및 사용법 소개 (K-RAG 패키지 컨셉 기반)

실제 `krag` 패키지 대신, 핵심 개념을 보여주는 간단한 예시 데이터와 설명으로 대체함.

In [ ]:
from langchain_core.documents import Document

# 각 쿼리에 대한 정답 문서 (실제 관련 문서)
actual_docs_sample = [
    # Query 1에 대한 정답
    [Document(metadata={'id': 'doc1'}, page_content='내용1')],
    # Query 2에 대한 정답
    [Document(metadata={'id': 'doc2'}, page_content='내용2'), Document(metadata={'id': 'doc5'}, page_content='내용5')]
]

# 각 쿼리에 대한 검색기가 예측한(찾아온) 문서
predicted_docs_sample = [
    # Query 1에 대한 예측
    [Document(metadata={'id': 'doc1'}, page_content='내용1'), Document(metadata={'id': 'doc3'}, page_content='내용3')],
    # Query 2에 대한 예측
    [Document(metadata={'id': 'doc4'}, page_content='내용4'), Document(metadata={'id': 'doc1'}, page_content='내용1'), 
     Document(metadata={'id': 'doc5'}, page_content='내용5'), Document(metadata={'id': 'doc2'}, page_content='내용2')]
]

print("샘플 정답 문서 (actual_docs_sample) 및 예측 문서 (predicted_docs_sample) 준비 완료.")

**평가 지표 설명 (개념)**

`(1) Hit Rate@k`
- 정의: 각 쿼리에 대해 검색된 상위 `k`개의 문서 중에 실제 정답 문서가 **하나라도 포함**되어 있으면 1 (Hit), 아니면 0으로 계산. 전체 쿼리에 대한 평균을 냄.
- 특징: 간단하고 직관적이지만, 정답 문서의 순위나 개수는 고려하지 않음.
- 예시 (k=2, Query 1): 예측 [`doc1`, `doc3`], 정답 [`doc1`]. `doc1`이 포함되어 Hit (1).
- 예시 (k=2, Query 2): 예측 [`doc4`, `doc1`], 정답 [`doc2`, `doc5`]. `doc2` 또는 `doc5` 미포함 (정답 문서 ID 기준). Not Hit (0). (샘플 데이터에서는 `doc5`가 3번째, `doc2`가 4번째에 있음)
- Hit Rate@2 = (1 + 0) / 2 = 0.5 (만약 Query 2 예측에 `doc5`나 `doc2`가 상위 2개 안에 있었다면 1)

`(2) MRR@k (Mean Reciprocal Rank)`
- 정의: 각 쿼리에 대해 **첫 번째 정답 문서**가 검색된 순위(rank)의 역수(1/rank)를 계산. 이 값들의 평균.
- 특징: 사용자가 원하는 결과를 얼마나 빨리 찾을 수 있는지(첫 정답의 순위)를 평가. 상위 `k`개 내에서만 고려.
- 예시 (k=4, Query 1): 예측 [`doc1`, `doc3`], 정답 [`doc1`]. `doc1`이 1등. RR = 1/1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`, `doc1`, `doc5`, `doc2`], 정답 [`doc2`, `doc5`]. 첫 정답 `doc5`가 3등. RR = 1/3.
- MRR@4 = (1 + 1/3) / 2 ≈ 0.667.

`(3) mAP@k (Mean Average Precision at k)`
- 정의: 각 쿼리에 대한 AP@k(Average Precision at k)를 구하고, 이들의 평균을 냄. AP@k는 상위 `k`개 결과 내에서 각 정답 문서가 나올 때마다의 Precision(정밀도) 값들의 평균.
- 특징: 검색 결과의 순서와 정확도를 모두 고려. 여러 정답이 있는 경우 유용.
- Precision@i: 상위 i개 결과 중 정답 문서의 비율.
- AP@k 계산: 각 정답 문서 위치 `j` (j <= k)에서의 Precision@j 값을 모두 더한 후, 총 정답 문서 수 (또는 k개 내에 있는 정답 문서 수)로 나눔.
- 예시 (k=4, Query 1): 예측 [`doc1`(정답), `doc3`], 정답 [`doc1`]. 
  - P@1 (`doc1`): 1/1 = 1. AP@4_Q1 = (1) / 1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`, `doc1`, `doc5`(정답), `doc2`(정답)], 정답 [`doc2`, `doc5`].
  - P@1 (`doc4`): 0/1=0.
  - P@2 (`doc1`): 0/2=0.
  - P@3 (`doc5`): 1/3. (첫 정답)
  - P@4 (`doc2`): 2/4. (두 번째 정답)
  - AP@4_Q2 = (1/3 + 2/4) / 2 = (0.333 + 0.5) / 2 = 0.4165.
- mAP@4 = (1 + 0.4165) / 2 ≈ 0.708.

`(4) NDCG@k (Normalized Discounted Cumulative Gain at k)`
- 정의: 검색 결과의 순서와 문서의 관련성 등급을 모두 고려하여 평가. 이상적인 검색 결과(IDCG) 대비 현재 검색 결과(DCG)의 비율.
- 특징: 관련성이 높은 문서가 상위에 있을수록 높은 점수. 가장 정교한 지표 중 하나.
- DCG@k = Σ ( (2^relevance_i - 1) / log2(i+1) )  (i는 순위 1부터 k까지, relevance_i는 i번째 문서의 관련도 점수)
- IDCG@k: 이상적인 순서일 때의 DCG@k (정답 문서들을 관련도 순으로 정렬했을 때).
- NDCG@k = DCG@k / IDCG@k.
- (이진 관련성 가정: 정답이면 1, 아니면 0)
- 예시 (k=4, Query 1): 예측 [`doc1`(rel=1), `doc3`(rel=0)], 정답 [`doc1`].
  - DCG@4_Q1 = (2^1-1)/log2(1+1) + (2^0-1)/log2(2+1) = 1/1 + 0 = 1.
  - IDCG@4_Q1 = (2^1-1)/log2(1+1) = 1.
  - NDCG@4_Q1 = 1/1 = 1.
- 예시 (k=4, Query 2): 예측 [`doc4`(0), `doc1`(0), `doc5`(1), `doc2`(1)], 정답 [`doc2`, `doc5`]. (이상적 순서는 `doc2`(1), `doc5`(1) 또는 그 반대)
  - DCG@4_Q2 = 0/log2(2) + 0/log2(3) + 1/log2(4) + 1/log2(5) = 0 + 0 + 1/2 + 1/2.32 = 0.5 + 0.43 = 0.93.
  - IDCG@4_Q2 = 1/log2(2) + 1/log2(3) = 1 + 1/1.58 = 1 + 0.63 = 1.63.
  - NDCG@4_Q2 = 0.93 / 1.63 ≈ 0.57.
- 평균 NDCG@4 = (1 + 0.57) / 2 = 0.785.

**실제 K-RAG 패키지는 이러한 계산을 자동화해줌.**

#### 테스트 데이터셋 평가 준비

평가 함수 `evaluate_qa_test`를 정의하고, 이전에 준비한 검색기들과 QA 데이터(`df_qa_test_for_eval`)를 사용하여 성능을 측정함.

In [ ]:
# 평가용 데이터프레임의 'context', 'source', 'doc_id' 컬럼은 문자열화된 리스트일 수 있음.
# 이를 실제 Document 객체 리스트로 변환하는 함수.
def actual_docs_from_df_row(df_row) -> List[Document]:
    # 문자열 리스트를 실제 리스트로 변환 (eval 사용은 신중해야 하나, 여기서는 생성 방식을 알고 있으므로 사용)
    # 안전한 방법은 ast.literal_eval(df_row['context']) 등을 사용하는 것임
    try:
        # 원본 노트북에서는 context, source, doc_id가 리스트 형태의 문자열로 저장되어 있었음
        # 예: "['문서 내용1']", "['출처1']", "['id1']"
        # pd.read_excel 시 자동으로 리스트로 파싱되지 않을 수 있음.
        # 만약 문자열 그대로라면 eval이나 ast.literal_eval 필요.
        # 여기서는 df_qa_test_for_eval이 이미 적절한 형태로 로드되었다고 가정하고 진행.
        # 실제로는 아래와 같이 처리 필요할 수 있음:
        # import ast
        # contexts = ast.literal_eval(df_row['context']) if isinstance(df_row['context'], str) else df_row['context']
        # sources = ast.literal_eval(df_row['source']) if isinstance(df_row['source'], str) else df_row['source']
        # doc_ids = ast.literal_eval(df_row['doc_id']) if isinstance(df_row['doc_id'], str) else df_row['doc_id']
        
        # 현재 df_qa_test_for_eval는 원본 노트북의 구조를 따르므로, 각 셀이 이미 리스트/문자열임.
        # context는 문자열 리스트, source도 문자열 리스트, doc_id도 문자열 리스트여야 함.
        # 그러나 원본에서는 하나의 context, source, doc_id가 리스트로 묶여 저장됨.
        # 즉, df_row['context']는 ['문서내용'] 형태임.

        # 원본 노트북의 의도대로라면, context, source, doc_id는 각각 단일 값의 리스트임.
        # 예: context = ['이것은 문서입니다.'], source = ['출처A'], doc_id = ['doc_xyz']
        # 여기서는 이 구조를 따른다고 가정하고, 해당 context가 정답 문서라고 간주.
        
        # context_content = df_row['context'][0] # 첫번째 (그리고 유일한) 컨텐츠
        # metadata = {'source': df_row['source'][0], 'id': df_row['doc_id'][0]} # id는 krag 호환을 위해 사용
        # return [Document(page_content=context_content, metadata=metadata)]
        
        # 원본 노트북의 context_to_document 함수와 유사하게, df_row의 'context', 'source', 'doc_id'가
        # 여러 개의 아이템을 가진 리스트 형태의 문자열이라고 가정하고 파싱 (예: "['컨텍스트1', '컨텍스트2']")
        import ast
        contexts = ast.literal_eval(str(df_row['context']))
        sources = ast.literal_eval(str(df_row['source']))
        doc_ids_from_df = ast.literal_eval(str(df_row['doc_id']))

        actual_docs_list = []
        for c, s, d_id in zip(contexts, sources, doc_ids_from_df):
            actual_docs_list.append(Document(page_content=c, metadata={'source': s, 'id': d_id})) # krag 호환 'id'
        return actual_docs_list

    except Exception as e:
        print(f"actual_docs_from_df_row 파싱 오류: {e} (입력값: {df_row})")
        return [] # 오류 시 빈 리스트 반환

if not df_qa_test_for_eval.empty:
    print("첫 번째 QA 데이터 행으로 정답 문서(actual_docs) 변환 테스트:")
    sample_actual_docs = actual_docs_from_df_row(df_qa_test_for_eval.iloc[0])
    # for doc_ in sample_actual_docs:
    #     print(f"  Content: {doc_.page_content[:50]}... Metadata: {doc_.metadata}")
    print(f"  변환된 문서 수: {len(sample_actual_docs)}")
    if sample_actual_docs:
        print(f"  첫 문서 메타데이터: {sample_actual_docs[0].metadata}")

In [ ]:
# !pip install krag # K-RAG 패키지 설치 (실제 사용 시)
from krag.evaluators import OfflineRetrievalEvaluators # K-RAG 평가 도구 (설치 필요)
from langchain_core.retrievers import BaseRetriever

def evaluate_retriever_performance(df_qa: pd.DataFrame, retriever: BaseRetriever, k_values: List[int]) -> pd.DataFrame:
    """
    주어진 QA 데이터프레임과 검색기를 사용하여 여러 k 값에 대한 IR 평가지표를 계산함.
    df_qa: 'question', 'context', 'source', 'doc_id' 컬럼을 가진 데이터프레임.
    retriever: LangChain BaseRetriever 구현체.
    k_values: 평가할 k 값들의 리스트 (예: [1, 3, 5]).
    """
    all_actual_docs = []
    all_predicted_docs = []
    
    print(f"총 {len(df_qa)}개의 질문에 대해 평가를 시작합니다...")
    for idx, row in df_qa.iterrows():
        question = row['question']
        actual_docs_for_query = actual_docs_from_df_row(row) # 해당 질문의 정답 문서들
        all_actual_docs.append(actual_docs_for_query)
        
        # 검색기 호출 시 k 값을 설정할 수 있다면 retriever의 search_kwargs를 수정해야 함.
        # EnsembleRetriever 등은 invoke 시 k를 직접 받지 않음. 초기 설정 k를 따름.
        # 여기서는 retriever가 내부적으로 최대 k개 (예: 5개)를 가져온다고 가정하고,
        # OfflineRetrievalEvaluators가 k_values에 따라 잘라서 평가한다고 가정.
        # (만약 retriever.search_kwargs['k'] 수정이 필요하다면 여기서 해야 함)
        predicted_docs_for_query = retriever.invoke(question)
        all_predicted_docs.append(predicted_docs_for_query)
        
        if (idx + 1) % 50 == 0:
            print(f"  {idx+1}/{len(df_qa)} 처리 완료...")

    print("모든 질문에 대한 예측 완료. 평가지표 계산 시작...")
    evaluator = OfflineRetrievalEvaluators(
        actual_docs=all_actual_docs,
        predicted_docs=all_predicted_docs
    )

    results = []
    for k_val in k_values:
        # OfflineRetrievalEvaluators는 k를 각 메소드 호출 시 받음
        hit_rate = evaluator.calculate_hit_rate(k=k_val)['hit_rate']
        mrr = evaluator.calculate_mrr(k=k_val)['mrr']
        map_score = evaluator.calculate_map(k=k_val)['map']
        ndcg = evaluator.calculate_ndcg(k=k_val)['ndcg']
        results.append({'k': k_val, 'HitRate': hit_rate, 'MRR': mrr, 'MAP': map_score, 'NDCG': ndcg})
        print(f"K={k_val} -> HitRate: {hit_rate:.3f}, MRR: {mrr:.3f}, MAP: {map_score:.3f}, NDCG: {ndcg:.3f}")
        
    return pd.DataFrame(results)

`- Kiwi 토크나이저 + BM25 검색기 평가`

  - **장점 (BM25+Kiwi):** 한국어에 대해 단순 BM25보다 높은 성능 기대, 계산 비용 상대적 낮음.
  - **단점:** 의미론적 이해 부족 한계는 여전함, Kiwi 모델 및 사용자 사전 관리 필요.

In [ ]:
# krag.retrievers.KiWiBM25RetrieverWithScore는 krag 패키지에만 있음.
# 여기서는 LangChain의 BM25Retriever (bm25_retriever_kiwi)를 사용.
# BM25Retriever는 점수를 직접 반환하지 않으므로, K-RAG의 Evaluator가 순서 기반으로 평가함.

# bm25_retriever_kiwi는 이미 생성되어 있다고 가정 (3.2 (2) 단계).
# 만약 bm25_retriever_kiwi.k 같은 속성이 없다면, from_documents 시 k 지정 또는
# retriever의 search_kwargs를 설정해야 하지만, BM25Retriever는 그런 인터페이스가 없음.
# BM25Retriever는 기본적으로 모든 문서를 대상으로 점수를 매기고 정렬함.
# OfflineRetrievalEvaluator가 상위 k개만 잘라서 평가함.

if bm25_retriever_kiwi and not df_qa_test_for_eval.empty:
    print("Kiwi + BM25 검색기 성능 평가 시작...")
    # BM25Retriever의 k 파라미터는 from_documents 시점에 설정됨. invoke 시점에는 변경 불가.
    # 여기서는 bm25_retriever_kiwi가 이미 모든 문서를 대상으로 점수화한다고 가정하고,
    # OfflineRetrievalEvaluators가 k_values에 따라 잘라서 평가함.
    # 만약 BM25Retriever.from_documents(..., k=N)으로 생성했다면 그 N이 최대 반환 개수.
    # Langchain BM25Retriever는 k를 생성시 받음.
    # results_bm25_kiwi = evaluate_retriever_performance(df_qa_test_for_eval, bm25_retriever_kiwi, k_values=[1, 2, 3, 5])
    # display(results_bm25_kiwi)
    print("BM25Retriever는 from_documents() 시 k를 설정해야 합니다.")
    print("이전 코드(3.2 (2))에서는 BM25Retriever.from_documents()에 k를 전달하지 않았으므로, 모든 문서를 반환할 수 있습니다.")
    print("평가 함수는 검색기가 반환한 결과 중 상위 k개만 사용합니다.")
    print("실제 평가 코드 실행은 주석 처리합니다 (K-RAG 설치 및 API 키 등 환경 의존성 때문).")
    
    # 예시: bm25_retriever_kiwi_k5 = BM25Retriever.from_documents(korean_docs, preprocess_func=kiwi_bm25_process_func, k=5)
    # results_bm25_kiwi = evaluate_retriever_performance(df_qa_test_for_eval, bm25_retriever_kiwi_k5, k_values=[1, 2, 3, 5])
    # display(results_bm25_kiwi)
else:
    print("Kiwi+BM25 검색기 또는 평가 데이터가 준비되지 않아 평가를 스킵합니다.")

`- Chroma 벡터저장소 검색기 (의미론적 검색) 평가`

  - **장점 (Chroma + BGE-M3):** 강력한 의미론적 검색 능력, 다양한 검색 옵션(MMR, 필터 등).
  - **단점:** 임베딩 및 검색에 계산 비용 발생, 특정 키워드 매칭에 약할 수 있음.

In [ ]:
# chroma_db_semantic (3.1 (1)에서 생성) 을 사용한 검색기
# chroma_k_retriever, chroma_threshold_retriever, chroma_mmr_retriever 등이 이미 있음.
# 여기서는 Top-K 검색기 (chroma_k_retriever)를 예시로 사용.
# 이 검색기는 생성 시 search_kwargs={'k': 2}로 설정됨.
# 평가 시 다양한 k를 보려면, retriever를 새로 만들거나 search_kwargs를 바꿔야 함.

if chroma_db_semantic and not df_qa_test_for_eval.empty:
    print("\nChroma (의미론적) 검색기 성능 평가 시작...")
    # 평가할 k_values에 맞춰 검색기 k를 설정하는 것이 좋음.
    # 여기서는 최대 k=5로 검색기를 만들고, 평가 함수가 잘라서 쓰도록 함.
    retriever_chroma_for_eval = chroma_db_semantic.as_retriever(search_kwargs={"k": 5})
    
    # results_chroma_semantic = evaluate_retriever_performance(df_qa_test_for_eval, retriever_chroma_for_eval, k_values=[1, 2, 3, 5])
    # display(results_chroma_semantic)
    print("실제 평가 코드 실행은 주석 처리합니다.")
else:
    print("Chroma DB (semantic) 또는 평가 데이터가 준비되지 않아 평가를 스킵합니다.")

`- Ensemble Retriever (하이브리드 검색) 평가`

  - **장점:** 의미론적 검색과 키워드 검색의 장점 결합으로 전반적인 성능 향상 기대.
  - **단점:** 구성 복잡, 가중치 등 하이퍼파라미터 튜닝 필요, 계산 비용 가장 높음.

In [ ]:
# ensemble_retriever (3.3에서 생성) 사용
# 이 retriever는 내부 retriever들의 k값을 따름. weights로 최종 결과 조합.
# 내부 retriever들이 충분한 후보군을 반환하도록 k 설정 필요.

if ensemble_retriever and not df_qa_test_for_eval.empty:
    print("\nEnsemble (하이브리드) 검색기 성능 평가 시작...")
    # ensemble_retriever의 내부 retriever들의 k가 평가하려는 k_values의 max보다 크거나 같아야 함.
    # 예: chroma_threshold_retriever.search_kwargs['k'] = 5
    # bm25_retriever_kiwi는 k를 생성시 설정. (여기서는 k=5로 가정)
    
    # results_ensemble = evaluate_retriever_performance(df_qa_test_for_eval, ensemble_retriever, k_values=[1, 2, 3, 5])
    # display(results_ensemble)
    print("실제 평가 코드 실행은 주석 처리합니다.")
else:
    print("Ensemble Retriever 또는 평가 데이터가 준비되지 않아 평가를 스킵합니다.")